In [2]:
# import libraries

import pandas as pd
import numpy as np
import regex as re

# notebook settings 

pd.set_option('display.max_rows', 120)
pd.set_option('display.max_columns', None)
settings :  { "notebook.output.textLineLimit": 9999 }

In [6]:
df = pd.read_csv('pre-processed_new.csv')

df.head()

,R_fighter,B_fighter,R_odds,B_odds,R_ev,B_ev,date,Winner,title_bout,weight_class,gender,no_of_rounds,B_Stance,R_Stance,lose_streak_dif,win_streak_dif,longest_win_streak_dif,win_dif,loss_dif,total_round_dif,total_title_bout_dif,ko_dif,sub_dif,height_dif,reach_dif,age_dif,sig_str_dif,avg_sub_att_dif,avg_td_dif,constant_1,B_match_weightclass_rank,R_match_weightclass_rank,R_Pound-for-Pound_rank,B_Pound-for-Pound_rank,finish,finish_round,r_dec_odds,b_dec_odds,r_sub_odds,b_sub_odds,r_ko_odds,b_ko_odds,weight_diff,win_by_decision_diff
0,Eric Schafer,Jason Brilz,140.0,-160,140.000000,62.500000,2010-03-21,Blue,False,Light Heavyweight,MALE,3,Orthodox,Orthodox,0,0,0,-1,2,-1,0,0,-2,-10.16,-10.16,-2,16.000000,-0.666667,0.833333,1,NaN,NaN,NaN,NaN,U-DEC,3.0,NaN,NaN,NaN,NaN,NaN,NaN,-20,-1
1,Brandon Vera,Jon Jones,215.0,-235,215.000000,42.553191,2010-03-21,Blue,False,Light Heavyweight,MALE,3,Orthodox,Orthodox,0,0,-1,-4,3,-14,0,-4,0,2.54,20.32,10,8.272727,0.159091,1.954545,1,NaN,NaN,NaN,NaN,KO/TKO,1.0,NaN,NaN,NaN,NaN,NaN,NaN,25,0
2,Junior Dos Santos,Gabriel Gonzaga,-250.0,230,40.000000,230.000000,2010-03-21,Red,False,Heavyweight,MALE,3,Orthodox,Orthodox,0,-3,0,3,-3,10,1,1,2,-5.08,-2.54,-4,-12.050000,0.700000,1.200000,1,NaN,NaN,NaN,NaN,KO/TKO,1.0,NaN,NaN,NaN,NaN,NaN,NaN,-4,0
3,Alessio Sakara,James Irvin,-120.0,100,83.333333,100.000000,2010-03-21,Red,False,Middleweight,MALE,3,Orthodox,Orthodox,-1,-2,0,-1,1,-5,0,0,0,5.08,7.62,-3,-15.550000,0.125000,-1.000000,1,NaN,NaN,NaN,NaN,KO/TKO,1.0,NaN,NaN,NaN,NaN,NaN,NaN,-20,2
4,Mike Pierce,Julio Paulino,-420.0,335,23.809524,335.000000,2010-03-21,Red,False,Welterweight,MALE,3,Orthodox,Orthodox,1,0,-1,-1,1,-6,0,0,0,10.16,7.62,-5,-40.500000,0.000000,-3.500000,1,NaN,NaN,NaN,NaN,U-DEC,3.0,NaN,NaN,NaN,NaN,NaN,NaN,0,1


# Baseline

In [ ]:
b_df=df.drop(['R_fighter','B_fighter'])

In [7]:
df.isnull().sum()

R_fighter                      0
B_fighter                      0
R_odds                         1
B_odds                         0
R_ev                           1
B_ev                           0
date                           0
Winner                         0
title_bout                     0
weight_class                   0
gender                         0
no_of_rounds                   0
B_Stance                       2
R_Stance                       0
lose_streak_dif                0
win_streak_dif                 0
longest_win_streak_dif         0
win_dif                        0
loss_dif                       0
total_round_dif                0
total_title_bout_dif           0
ko_dif                         0
sub_dif                        0
height_dif                     0
reach_dif                      0
age_dif                        0
sig_str_dif                    0
avg_sub_att_dif                0
avg_td_dif                     0
constant_1                     0
B_match_we

In [8]:
df.describe()

,R_odds,B_odds,R_ev,B_ev,no_of_rounds,lose_streak_dif,win_streak_dif,longest_win_streak_dif,win_dif,loss_dif,total_round_dif,total_title_bout_dif,ko_dif,sub_dif,height_dif,reach_dif,age_dif,sig_str_dif,avg_sub_att_dif,avg_td_dif,constant_1,B_match_weightclass_rank,R_match_weightclass_rank,R_Pound-for-Pound_rank,B_Pound-for-Pound_rank,finish_round,r_dec_odds,b_dec_odds,r_sub_odds,b_sub_odds,r_ko_odds,b_ko_odds,weight_diff,win_by_decision_diff
count,4538.000000,4539.000000,4538.000000,4539.000000,4539.000000,4539.000000,4539.000000,4539.000000,4539.000000,4539.000000,4539.000000,4539.000000,4539.000000,4539.000000,4539.000000,4539.000000,4539.000000,4539.000000,4539.000000,4539.000000,4539.0,790.000000,1202.000000,151.000000,34.000000,3927.000000,3821.000000,3808.000000,3589.000000,3577.000000,3589.000000,3576.000000,4539.000000,4539.000000
mean,-117.605553,65.640229,94.601874,166.547600,3.174708,0.097158,-0.184622,-0.752368,-1.368804,0.337960,-5.088125,-0.274289,-0.468826,-0.270104,0.021172,-0.262412,0.298965,-3.694229,-0.063034,-0.210209,1.0,8.336709,6.922629,6.589404,9.352941,2.414057,294.095001,414.601103,836.545277,1055.214425,517.899136,652.522092,0.437762,0.608724
std,267.862385,246.955440,82.452264,136.364516,0.560657,0.979309,1.682571,1.995680,3.904903,2.744744,16.439217,1.492120,1.944035,1.730323,7.040320,9.505267,5.185567,22.606399,0.867129,1.721720,0.0,4.240315,4.646523,4.063285,4.291516,0.991398,230.361088,304.652550,548.162720,626.734296,413.558169,460.338676,9.501910,2.214435
min,-1700.000000,-1200.000000,5.882353,8.333333,3.000000,-5.000000,-13.000000,-12.000000,-28.000000,-18.000000,-448.000000,-14.000000,-21.000000,-13.000000,-187.960000,-187.960000,-17.000000,-118.000000,-8.300000,-11.000000,1.0,1.000000,0.000000,1.000000,2.000000,1.000000,-440.000000,-200.000000,-370.000000,-1250.000000,-550.000000,-275.000000,-75.000000,-10.000000
25%,-250.000000,-145.000000,40.000000,68.965517,3.000000,0.000000,-1.000000,-2.000000,-3.000000,-1.000000,-11.000000,0.000000,-1.000000,-1.000000,-5.080000,-5.080000,-3.000000,-14.636243,-0.440000,-1.000000,1.0,5.000000,3.000000,3.000000,5.000000,1.000000,167.000000,225.000000,430.000000,585.000000,242.000000,325.000000,0.000000,0.000000
50%,-150.000000,130.000000,66.666667,130.000000,3.000000,0.000000,0.000000,-1.000000,-1.000000,0.000000,-3.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-0.750000,0.000000,0.000000,1.0,9.000000,7.000000,6.000000,10.000000,3.000000,250.000000,347.500000,701.000000,950.000000,439.000000,550.000000,0.000000,0.000000
75%,125.000000,220.000000,125.000000,220.000000,3.000000,0.000000,0.000000,0.000000,0.000000,2.000000,2.000000,0.000000,0.000000,0.000000,5.080000,5.080000,4.000000,4.012987,0.237424,0.600000,1.0,12.000000,11.000000,10.000000,13.000000,3.000000,395.000000,525.000000,1190.000000,1400.000000,700.000000,900.000000,0.000000,1.000000
max,775.000000,1300.000000,775.000000,1300.000000,5.000000,5.000000,9.000000,14.000000,23.000000,16.000000,80.000000,14.000000,14.000000,10.000000,30.480000,30.480000,17.000000,128.222222,6.000000,10.860000,1.0,15.000000,15.000000,15.000000,15.000000,5.000000,2200.000000,2600.000000,4665.000000,4785.000000,2675.000000,3200.000000,80.000000,12.000000
